![](https://cdn.cms-twdigitalassets.com/content/blog-twitter/engineering/en_us/topics/open-source/2017/Introducing-Torch-Decision-Trees/_jcr_content/par/rail-blog-container/column/image.img.png/1507323277313.png)blog.twitter.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Code By Alin Cijov  https://www.kaggle.com/alincijov/credit-card-pytorch-decision-tree/comments

In [ ]:
import keras
from keras import regularizers, optimizers
from keras import losses
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, Dropout, Embedding, LSTM
from keras.optimizers import RMSprop, Adam, Nadam
from keras.preprocessing import sequence

import torch
from torch import nn
from functools import reduce

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import tensorflow
import sys

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadsgermancsv/german.csv', delimiter=';', encoding = "ISO-8859-2", nrows = nRowsRead)
df.dataframeName = 'german.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

In [ ]:
# Standardize
df['Account_Balance'] = StandardScaler().fit_transform(df['Account_Balance'].values.reshape(-1, 1))
df['Duration_of_Credit_monthly'] = StandardScaler().fit_transform(df['Duration_of_Credit_monthly'].values.reshape(-1, 1))

In [ ]:
anomalies = df[df["Creditability"] == 1]
normal = df[df["Creditability"] == 0]

anomalies.shape, normal.shape

In [ ]:
for f in range(0, 20):
    normal = normal.iloc[np.random.permutation(len(normal))]
    

data_set = pd.concat([normal[:2000], anomalies])

x_train, x_test = train_test_split(data_set, test_size = 0.4, random_state = 42)

x_train = x_train.sort_values(by=['Duration_of_Credit_monthly'])
x_test = x_test.sort_values(by=['Duration_of_Credit_monthly'])

y_train = x_train["Creditability"]
y_test = x_test["Creditability"]

x_train.head(10)

In [ ]:
x_train = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1])
x_test = np.array(x_test).reshape(x_test.shape[0], x_test.shape[1])
input_shape = (x_train.shape[1], 1)

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

print("Shapes:\nx_train:%s\ny_train:%s\n" % (x_train.shape, y_train.shape))
print("x_test:%s\ny_test:%s\n" % (x_test.shape, y_test.shape))
print("input_shape:{}\n".format(input_shape))

In [ ]:
class Tree():
    def __init__(self):
        self.num_cut = [1, 1]
        self.num_leaf = np.prod(np.array(self.num_cut) + 1)
        self.num_class = 2
        
    def torch_kron_prod(self, a, b):
        res = torch.einsum('ij,ik->ijk', [a, b])
        res = torch.reshape(res, [-1, np.prod(res.shape[1:])])
        return res
    
    def torch_bin(self, x, cut_points, temperature=0.1):
        D = cut_points.shape[0]
        W = torch.reshape(torch.linspace(1.0, D + 1.0, D + 1), [1, -1])
        cut_points, _ = torch.sort(cut_points)
        b = torch.cumsum(torch.cat([torch.zeros([1]), -cut_points], 0),0)
        h = torch.matmul(x, W) + b
        res = torch.exp(h-torch.max(h))
        res = res/torch.sum(res, dim=-1, keepdim=True)
        return h
    
    def nn_decision_tree(self, x, cut_points_list, leaf_score, temperature=0.1):
        leaf = reduce(self.torch_kron_prod,
                      map(lambda z: self.torch_bin(x[:, z[0]:z[0] + 1], z[1], temperature), enumerate(cut_points_list)))
        return torch.matmul(leaf, leaf_score)

In [ ]:
tree = Tree()

cut_points_list = [torch.rand([i], requires_grad=True) for i in tree.num_cut]
leaf_score = torch.rand([tree.num_leaf, tree.num_class], requires_grad=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cut_points_list + [leaf_score], lr=0.001, weight_decay=0.001)

#Numbers below were taken from shapes. (Input 8)  Shapes:
x_train:(600, 21)
y_train:(600, 2)

So I wrote 21 (columns number) and 2 (last number). The 16 and 8, I just copied from Alin Cijov. Have no clue about them.

https://www.kaggle.com/alincijov/credit-card-pytorch-decision-tree/comments

In [ ]:
model = nn.Sequential(
          nn.Linear(21,16),
          nn.ReLU(),
          nn.Linear(16,8),
          nn.ReLU(),
          nn.Linear(8,2),
          nn.Sigmoid()
        )

In [ ]:
for i in range(300):
    optimizer.zero_grad()
    x_batches = torch.split(torch.tensor(x_train).type(torch.float32), 32)
    y_batches = torch.split(torch.tensor(y_train).type(torch.long), 32)
    
    losses = torch.zeros(len(x_batches))
    accs = torch.zeros(len(x_batches))
    for j,x in enumerate(x_batches):
        out = model(x)
        y_pred = tree.nn_decision_tree(out, cut_points_list, 
                                       leaf_score, temperature=0.1)
        y_max = torch.max(y_batches[j], axis=1)[1]
        y_pred_max = torch.max(y_pred, axis=1)[1]
        
        acc = len(torch.where(y_max == y_pred_max)[0]) / len(y_max)
        accs[j] = acc
        
        loss = loss_function(y_pred, y_max)
        losses[j] = loss
        
        loss.backward()
        optimizer.step()
    if((i+1) % 20 == 0):
        print("i:{:4d}, loss:{:1.3f}, acc:{:1.3f}".format(i+1, losses.mean(), accs.mean()))

In [ ]:
def predict(tree, X, y):
    X = torch.tensor(X).type(torch.float32)
    y = torch.tensor(y).type(torch.long)

    out = model(X)
    y_pred = tree.nn_decision_tree(out, cut_points_list, leaf_score, temperature=0.1)
    y_max = torch.max(y, axis=1)[1]
    y_pred_max = torch.max(y_pred, axis=1)[1]

    acc = len(torch.where(y_max == y_pred_max)[0]) / len(y_max)
    loss = loss_function(y_pred, y_max)
    return y_pred, acc, loss

In [ ]:
y_pred, acc, loss = predict(tree, x_test, y_test)
print("Accuracy:{:1.3f}, Loss:{:1.3f}"
         .format(acc, loss))

In [ ]:
class Visualization:
    labels = ["Normal", "Anomaly"]

    def draw_confusion_matrix(self, y, ypred):
        matrix = confusion_matrix(y, ypred)

        plt.figure(figsize=(10, 8))
        colors=[ "pink","purple"]
        sns.heatmap(matrix, xticklabels=self.labels, yticklabels=self.labels, cmap=colors, annot=True, fmt="d")
        plt.title("Confusion Matrix")
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.show()


    def draw_anomaly(self, y, error, threshold):
        groupsDF = pd.DataFrame({'error': error,
                                 'true': y}).groupby('true')

        figure, axes = plt.subplots(figsize=(12, 8))

        for name, group in groupsDF:
            axes.plot(group.index, group.error, marker='x' if name == 1 else 'o', linestyle='',
                    color='r' if name == 1 else 'g', label="Anomaly" if name == 1 else "Normal")

        axes.hlines(threshold, axes.get_xlim()[0], axes.get_xlim()[1], colors="b", zorder=100, label='Threshold')
        axes.legend()
        
        plt.title("Anomalies")
        plt.ylabel("Error")
        plt.xlabel("Data")
        plt.show()

    def draw_error(self, error, threshold):
            plt.plot(error, marker='o', ms=3.5, linestyle='',
                     label='Point')

            plt.hlines(threshold, xmin=0, xmax=len(error)-1, colors="b", zorder=100, label='Threshold')
            plt.legend()
            plt.title("Reconstruction error")
            plt.ylabel("Error")
            plt.xlabel("Data")
            plt.show()

In [ ]:
visualize = Visualization()
y_pred2 = torch.max(y_pred, axis=1)[1].detach().numpy()
y_test2 = np.argmax(y_test, axis=1)
visualize.draw_confusion_matrix(y_test2, y_pred2)

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thank you Alin Cijov for all the code' )